<a href="https://colab.research.google.com/github/shyakx/finance_chatbot/blob/main/Updated_Finance_Chatbot_T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 Finance Domain Chatbot using T5 on Colab

In [1]:
!pip install -q transformers datasets pandas gradio

In [2]:

import pandas as pd
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from datasets import Dataset


## 📥 Load and Clean Dataset

In [3]:

# Upload the dataset manually via the file upload button on the left in Colab
df = pd.read_csv('/content/Financial-QA-10k.csv')
df.dropna(subset=['question', 'answer', 'context'], inplace=True)
df = df[['question', 'context', 'answer']]


## 🧹 Format Data for T5

In [4]:

def format_for_t5(example):
    return {
        'input_text': f"question: {example['question']} context: {example['context']}",
        'target_text': example['answer']
    }

formatted = df.apply(format_for_t5, axis=1, result_type='expand')
dataset = Dataset.from_pandas(formatted)


## 🔤 Tokenization

In [5]:

tokenizer = T5Tokenizer.from_pretrained('t5-small')

def tokenize_data(example):
    input_enc = tokenizer(example['input_text'], truncation=True, padding='max_length', max_length=512)
    target_enc = tokenizer(example['target_text'], truncation=True, padding='max_length', max_length=64)
    input_enc['labels'] = target_enc['input_ids']
    return input_enc

tokenized_dataset = dataset.map(tokenize_data, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(['input_text', 'target_text'])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/6997 [00:00<?, ? examples/s]

## 📦 Load Model

In [6]:

model = T5ForConditionalGeneration.from_pretrained('t5-small')
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

## 🏋️ Training Setup

In [7]:

training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=4,
    num_train_epochs=2,
    save_steps=500,
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=100
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()


/tmp/ipython-input-7-2010900407.py:11: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: s-shyaka1 (j-chemirmir-glasgow-caledonian-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
100,3.828200
200,0.668800
300,0.588000
400,0.550400
500,0.485900
600,0.439300
700,0.482300
800,0.571800
900,0.455100
1000,0.412900


TrainOutput(global_step=3500, training_loss=0.5596324005126954, metrics={'train_runtime': 705.7082, 'train_samples_per_second': 19.83, 'train_steps_per_second': 4.96, 'total_flos': 1893973169799168.0, 'train_loss': 0.5596324005126954, 'epoch': 2.0})

## 💾 Save Model

In [8]:

model.save_pretrained("/content/finbuddy-model")
tokenizer.save_pretrained("/content/finbuddy-model")


('/content/finbuddy-model/tokenizer_config.json',
 '/content/finbuddy-model/special_tokens_map.json',
 '/content/finbuddy-model/spiece.model',
 '/content/finbuddy-model/added_tokens.json')

## 💬 Inference Function

In [13]:
def chat_with_bot(query, context):
    input_text = f"question: {query} context: {context}"

    # Move input to same device as model
    device = model.device
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

    # Generate output
    output_ids = model.generate(input_ids, max_length=100)

    # Decode and return response
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# ✅ Sample usage
chat_with_bot(
    "What is a derivative?",
    "In finance, a derivative is a contract whose value is based on an underlying asset."
)


'A derivative is a contract whose value is based on an underlying asset.'